In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import datetime
from datetime import timedelta
from datetime import datetime

In [ ]:
CURRENT_LOCATION = "bolz"

In [ ]:
path_to_file = 'APPA_data_merged.csv'
df = pd.read_csv(path_to_file, parse_dates=["Data"])
df = df[df.Stazione == "Via Bolzano"]
df = df[df.Inquinante == "PM10"]

Makes the daily mean from the hour values

In [ ]:

df_daily = df.resample("D", on='Data').mean()
df_daily = df_daily.reset_index()

In [ ]:
df_daily.head()

In [ ]:
weather_file_path = "weather_data.csv"
weather_df = pd.read_csv(weather_file_path, parse_dates=["Data"])

In [ ]:
locations = list(set(weather_df.columns[1:].map(lambda x: x.split()[0])))
locations

In [ ]:
weather_loc = {}

for location in locations:
    location_columns = weather_df.columns[
        weather_df.columns.map(lambda x: x.split()[0] == location)
    ]

    location_df = weather_df[["Data"] + location_columns.tolist()]
    location_df.columns = location_df.columns.str.replace(location, "").str.strip()
    location_df["Location"] = location
    
    weather_loc[location] = location_df


In [ ]:
df_weather_long = pd.concat(weather_loc.values())
# df_weather_long.to_csv("./weather_long_format.csv", index=None)

In [319]:
merged_df = df_daily.join(
    df_weather_long[df_weather_long.Location == CURRENT_LOCATION].set_index("Data"), 
    on="Data"
)
merged_df

,Data,Valore,rain,temp,Location,hum,w_dir,w_sp,press,sol
0,2012-01-01,55.727273,0.0,0.0,bolz,78.6,342.0,0.7,995.5,6433.5
1,2012-01-02,62.291667,15.0,-0.4,bolz,95.0,348.0,1.0,994.7,1154.3
2,2012-01-03,36.708333,0.6,3.6,bolz,86.8,329.0,1.8,997.5,7082.0
3,2012-01-04,33.041667,0.0,0.1,bolz,90.0,337.0,1.0,993.7,3149.2
4,2012-01-05,26.375000,0.0,1.0,bolz,79.8,343.0,1.5,980.3,5839.9
...,...,...,...,...,...,...,...,...,...,...
3837,2022-07-04,16.250000,7.0,23.2,bolz,70.6,315.0,3.7,991.4,16440.8
3838,2022-07-05,9.291667,0.0,24.9,bolz,59.5,330.0,2.9,989.0,25228.8
3839,2022-07-06,9.875000,0.0,25.2,bolz,54.2,336.0,2.0,990.8,28751.0
3840,2022-07-07,14.875000,4.0,23.5,bolz,62.2,296.0,1.3,989.6,19994.8


In [ ]:
# add column for the value of the previous day

# prev_day_values = []
# for date in merged_df["Data"]:
#     prev_day_value = date - timedelta(hours=int(24))
#     series = merged_df[merged_df.Data == prev_day_value].Valore.values.tolist()
#     if len(series) > 0:
#         value = series[0]
#     else:
#         value = merged_df[merged_df.Data == date].Valore.values.tolist()[0]  # takes the current day, not the previous

#     prev_day_values.append(value)

# merged_df["Prev day"] = prev_day_values
# merged_df


In [314]:
merged_df.isna().sum()

Data         0
Valore      13
rain         5
temp         5
Location     0
hum          3
w_dir        3
w_sp         4
press        3
sol          4
dtype: int64

In [315]:
merged_df.corr()

,Valore,rain,temp,hum,w_dir,w_sp,press,sol
Valore,1.000000,-0.122648,-0.444380,0.274519,0.004813,-0.486611,0.297860,-0.391982
rain,-0.122648,1.000000,0.016624,0.373602,0.027997,-0.001040,-0.321844,-0.245043
temp,-0.444380,0.016624,1.000000,-0.251255,-0.096029,0.401969,-0.147064,0.733091
hum,0.274519,0.373602,-0.251255,1.000000,0.104384,-0.648380,-0.133332,-0.596991
w_dir,0.004813,0.027997,-0.096029,0.104384,1.000000,-0.129086,0.102721,-0.053515
w_sp,-0.486611,-0.001040,0.401969,-0.648380,-0.129086,1.000000,-0.231920,0.492839
press,0.297860,-0.321844,-0.147064,-0.133332,0.102721,-0.231920,1.000000,0.052634
sol,-0.391982,-0.245043,0.733091,-0.596991,-0.053515,0.492839,0.052634,1.000000


In [316]:
merged_df.describe()

,Valore,rain,temp,hum,w_dir,w_sp,press,sol
count,3829.000000,3837.000000,3837.000000,3839.000000,3839.000000,3838.000000,3839.000000,3838.000000
mean,24.245645,2.794423,12.800469,70.803204,299.088304,1.738093,989.457333,13425.558494
std,14.101629,7.923554,8.031611,15.276657,68.967493,0.802883,7.334207,8128.025217
min,0.000000,0.000000,-6.300000,18.600000,0.000000,0.000000,952.700000,0.000000
25%,14.291667,0.000000,5.900000,60.300000,294.500000,1.100000,985.200000,6420.425000
50%,21.333333,0.000000,13.100000,71.400000,324.000000,1.700000,989.400000,12330.800000
75%,30.958333,0.800000,19.800000,82.550000,338.000000,2.200000,993.700000,20257.000000
max,156.125000,97.000000,29.600000,100.000000,360.000000,6.700000,1014.900000,31686.900000


In [318]:
merged_df = merged_df.dropna()
merged_df

,Data,Valore,rain,temp,Location,hum,w_dir,w_sp,press,sol
0,2012-01-01,55.727273,0.0,0.0,bolz,78.6,342.0,0.7,995.5,6433.5
1,2012-01-02,62.291667,15.0,-0.4,bolz,95.0,348.0,1.0,994.7,1154.3
2,2012-01-03,36.708333,0.6,3.6,bolz,86.8,329.0,1.8,997.5,7082.0
3,2012-01-04,33.041667,0.0,0.1,bolz,90.0,337.0,1.0,993.7,3149.2
4,2012-01-05,26.375000,0.0,1.0,bolz,79.8,343.0,1.5,980.3,5839.9
...,...,...,...,...,...,...,...,...,...,...
3837,2022-07-04,16.250000,7.0,23.2,bolz,70.6,315.0,3.7,991.4,16440.8
3838,2022-07-05,9.291667,0.0,24.9,bolz,59.5,330.0,2.9,989.0,25228.8
3839,2022-07-06,9.875000,0.0,25.2,bolz,54.2,336.0,2.0,990.8,28751.0
3840,2022-07-07,14.875000,4.0,23.5,bolz,62.2,296.0,1.3,989.6,19994.8


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
SEED = 42

In [ ]:
input_columns = [
    'rain', 
    'temp', 
    'hum', 
    'w_dir', 
    'w_sp', 
    'press', 
    'sol',
    #'Prev day'
]

y = merged_df['Valore']
X = merged_df[input_columns]

In [ ]:
test_years = [2015, 2017]
train_years = [2012, 2013, 2014, 2016, 2018, 2019, 2020, 2021, 2022]


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, 
#                                                     test_size=0.2, 
#                                                     random_state=SEED)

In [ ]:
X_train, y_train, X_test, y_test = (
    X[merged_df.Data.dt.year.isin(train_years)],
    y[merged_df.Data.dt.year.isin(train_years)],
    X[merged_df.Data.dt.year.isin(test_years)],
    y[merged_df.Data.dt.year.isin(test_years)]
)

In [ ]:
X.shape

In [ ]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

In [ ]:
regressor.intercept_

In [ ]:
regressor.coef_

In [ ]:
feature_names = X.columns
model_coefficients = regressor.coef_

coefficients_df = pd.DataFrame(data = model_coefficients, 
                              index = feature_names, 
                              columns = ['Coefficient value'])
print(coefficients_df)

In [ ]:
feature_names = X.columns
model_coefficients = regressor.coef_

coefficients_df = pd.DataFrame(data = model_coefficients, 
                              index = feature_names, 
                              columns = ['Coefficient value'])
print(coefficients_df)

In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(results)

In [ ]:
results.corr()

In [ ]:
test_chart_data = merged_df[
    merged_df.Data.dt.year.isin(test_years)
]
test_chart_data["pred"] = regressor.predict(test_chart_data[input_columns])



In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
test_chart_data.plot("Data", "Valore", ax=ax)
test_chart_data.plot("Data", "pred", ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
test_chart_data.plot.scatter("Valore", "pred", alpha=0.3, ax=ax)
# ax.set_xlim(-5, 165)
# ax.set_ylim(-5, 165)

ax.set_xlim(-5, 105)
ax.set_ylim(-5, 105)



In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import numpy as np

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'Mean absolute error: {mae:.2f}')
print(f'Mean squared error: {mse:.2f}')
print(f'Root mean squared error: {rmse:.2f}')

In [ ]:
regressor.score(X_test, y_test)

In [ ]:
actual_minus_predicted = sum((y_test - y_pred)**2)
actual_minus_actual_mean = sum((y_test - y_test.mean())**2)
r2 = 1 - actual_minus_predicted/actual_minus_actual_mean
print('R²:', r2)